# Standard .NET Interfaces - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **trivial** implementations of standard .NET interfaces including IEnumerable, IComparable, IComparer, IEquatable, IEnumerator, and others.

## IEquatable&lt;T&gt; Interface

### ✅ Good Example - Meaningful IEquatable Implementation

In [ ]:
// ✅ GOOD: Product class with meaningful equality comparison
public class Product : IEquatable<Product>
{
    public string SKU { get; }
    public string Name { get; set; }
    public decimal Price { get; set; }
    public string Category { get; set; }
    
    public Product(string sku, string name, decimal price, string category)
    {
        SKU = sku ?? throw new ArgumentNullException(nameof(sku));
        Name = name ?? throw new ArgumentNullException(nameof(name));
        Price = price;
        Category = category;
    }
    
    // Business logic: Products are equal if they have the same SKU
    // This makes sense because SKU is unique identifier in business context
    public bool Equals(Product other)
    {
        if (other is null) return false;
        if (ReferenceEquals(this, other)) return true;
        return string.Equals(SKU, other.SKU, StringComparison.OrdinalIgnoreCase);
    }
    
    public override bool Equals(object obj) => Equals(obj as Product);
    
    public override int GetHashCode() => SKU?.ToLowerInvariant().GetHashCode() ?? 0;
    
    public static bool operator ==(Product left, Product right) => 
        left?.Equals(right) ?? right is null;
    
    public static bool operator !=(Product left, Product right) => !(left == right);
    
    public override string ToString() => $"{Name} ({SKU}) - ${Price}";
}

In [ ]:
// Usage example - meaningful equality in business context
var product1 = new Product("ABC123", "Laptop", 999.99m, "Electronics");
var product2 = new Product("ABC123", "Gaming Laptop", 1299.99m, "Electronics"); // Same SKU, different details

Console.WriteLine($"Products equal: {product1.Equals(product2)}"); // True - same SKU
Console.WriteLine($"Using == operator: {product1 == product2}");   // True

// Useful in collections for deduplication
var products = new HashSet<Product> { product1, product2 };
Console.WriteLine($"Unique products: {products.Count}"); // 1 - deduplication by SKU

// Meaningful for finding products in inventory
var inventory = new List<Product> { product1 };
var searchProduct = new Product("ABC123", "", 0, "");
bool inStock = inventory.Contains(searchProduct); // True - found by SKU
Console.WriteLine($"Product in stock: {inStock}");

### ❌ Bad Example - Trivial IEquatable Implementation

In [ ]:
// ❌ BAD: Simple data class with default equality
public class BadProduct : IEquatable<BadProduct>
{
    public string Name { get; set; }
    public decimal Price { get; set; }
    
    // Meaningless equality - just comparing all properties
    // No business logic, same as default reference equality
    public bool Equals(BadProduct other)
    {
        return other != null && Name == other.Name && Price == other.Price;
    }
    
    public override bool Equals(object obj) => Equals(obj as BadProduct);
    public override int GetHashCode() => HashCode.Combine(Name, Price);
}

// This adds no value - could just use record type or default equality

## IComparable&lt;T&gt; Interface

### ✅ Good Example - Meaningful IComparable Implementation

In [ ]:
// ✅ GOOD: Employee with meaningful comparison for payroll processing
public class Employee : IComparable<Employee>, IEquatable<Employee>
{
    public int EmployeeId { get; }
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public decimal Salary { get; set; }
    public DateTime HireDate { get; set; }
    public int DepartmentId { get; set; }
    
    public Employee(int id, string firstName, string lastName, decimal salary, DateTime hireDate, int deptId)
    {
        EmployeeId = id;
        FirstName = firstName;
        LastName = lastName;
        Salary = salary;
        HireDate = hireDate;
        DepartmentId = deptId;
    }
    
    // Business rule: Sort employees by department first, then by hire date (seniority)
    // This is meaningful for organizational charts, layoff decisions, etc.
    public int CompareTo(Employee other)
    {
        if (other is null) return 1;
        
        // First compare by department
        int deptComparison = DepartmentId.CompareTo(other.DepartmentId);
        if (deptComparison != 0) return deptComparison;
        
        // Then by hire date (earlier hire date = higher seniority)
        int hireDateComparison = HireDate.CompareTo(other.HireDate);
        if (hireDateComparison != 0) return hireDateComparison;
        
        // Finally by ID for consistent ordering
        return EmployeeId.CompareTo(other.EmployeeId);
    }
    
    public bool Equals(Employee other) => other != null && EmployeeId == other.EmployeeId;
    public override bool Equals(object obj) => Equals(obj as Employee);
    public override int GetHashCode() => EmployeeId.GetHashCode();
    
    public override string ToString() => $"{FirstName} {LastName} (ID: {EmployeeId}, Dept: {DepartmentId}, Hired: {HireDate:yyyy-MM-dd})";
}

In [ ]:
// Usage example - meaningful sorting for business operations
var employees = new List<Employee>
{
    new Employee(1, "John", "Doe", 75000, new DateTime(2020, 3, 15), 2),
    new Employee(2, "Jane", "Smith", 85000, new DateTime(2019, 8, 22), 1),
    new Employee(3, "Bob", "Johnson", 65000, new DateTime(2021, 1, 10), 2),
    new Employee(4, "Alice", "Brown", 95000, new DateTime(2018, 5, 5), 1)
};

// Sort employees by business rules
employees.Sort();

Console.WriteLine("Employees sorted by department and seniority:");
foreach (var emp in employees)
{
    Console.WriteLine(emp);
}

// Useful for finding senior employee in department
var seniorInDept1 = employees.Where(e => e.DepartmentId == 1).Min();
Console.WriteLine($"\nMost senior in Dept 1: {seniorInDept1}");

### ❌ Bad Example - Trivial IComparable Implementation

In [ ]:
// ❌ BAD: Meaningless comparison
public class BadEmployee : IComparable<BadEmployee>
{
    public string Name { get; set; }
    public int Age { get; set; }
    
    // Arbitrary comparison with no business meaning
    public int CompareTo(BadEmployee other)
    {
        return other == null ? 1 : Name.CompareTo(other.Name);
    }
}

// Just sorting by name alphabetically has no business value
// Could use OrderBy() when needed instead

## IComparer&lt;T&gt; Interface

### ✅ Good Example - Meaningful IComparer Implementation

In [ ]:
// ✅ GOOD: Custom comparers for different business scenarios
public class TaskPriorityComparer : IComparer<WorkTask>
{
    // Business logic: High priority tasks first, then by deadline
    public int Compare(WorkTask x, WorkTask y)
    {
        if (x == null && y == null) return 0;
        if (x == null) return 1;
        if (y == null) return -1;
        
        // Compare by priority first (High = 3, Medium = 2, Low = 1)
        int priorityComparison = y.Priority.CompareTo(x.Priority); // Descending
        if (priorityComparison != 0) return priorityComparison;
        
        // Then by deadline (earlier deadlines first)
        return x.Deadline.CompareTo(y.Deadline);
    }
}

public class TaskCreationDateComparer : IComparer<WorkTask>
{
    // Different business scenario: Sort by creation date for audit reports
    public int Compare(WorkTask x, WorkTask y)
    {
        if (x == null && y == null) return 0;
        if (x == null) return 1;
        if (y == null) return -1;
        
        return x.CreatedDate.CompareTo(y.CreatedDate);
    }
}

public enum TaskPriority { Low = 1, Medium = 2, High = 3 }

public class WorkTask
{
    public string Title { get; set; }
    public TaskPriority Priority { get; set; }
    public DateTime Deadline { get; set; }
    public DateTime CreatedDate { get; set; }
    
    public WorkTask(string title, TaskPriority priority, DateTime deadline)
    {
        Title = title;
        Priority = priority;
        Deadline = deadline;
        CreatedDate = DateTime.Now;
    }
    
    public override string ToString() => $"{Title} (Priority: {Priority}, Deadline: {Deadline:yyyy-MM-dd})";
}

In [ ]:
// Usage example - different sorting strategies for different business needs
var tasks = new List<WorkTask>
{
    new WorkTask("Fix critical bug", TaskPriority.High, DateTime.Now.AddDays(1)),
    new WorkTask("Write documentation", TaskPriority.Low, DateTime.Now.AddDays(7)),
    new WorkTask("Code review", TaskPriority.Medium, DateTime.Now.AddDays(2)),
    new WorkTask("Security audit", TaskPriority.High, DateTime.Now.AddDays(3))
};

// Sort for daily work prioritization
var priorityComparer = new TaskPriorityComparer();
var workQueue = new List<WorkTask>(tasks);
workQueue.Sort(priorityComparer);

Console.WriteLine("Tasks sorted by priority and deadline (work queue):");
foreach (var task in workQueue)
{
    Console.WriteLine(task);
}

// Sort for audit reports
var auditComparer = new TaskCreationDateComparer();
var auditList = new List<WorkTask>(tasks);
auditList.Sort(auditComparer);

Console.WriteLine("\nTasks sorted by creation date (audit report):");
foreach (var task in auditList)
{
    Console.WriteLine($"{task} (Created: {task.CreatedDate:yyyy-MM-dd HH:mm})");
}

// Use with SortedSet for automatic ordering
var priorityQueue = new SortedSet<WorkTask>(priorityComparer);
foreach (var task in tasks) priorityQueue.Add(task);

Console.WriteLine($"\nAuto-sorted priority queue has {priorityQueue.Count} tasks");

### ❌ Bad Example - Trivial IComparer Implementation

In [ ]:
// ❌ BAD: Unnecessary comparer that duplicates built-in functionality
public class BadStringComparer : IComparer<string>
{
    // This is pointless - string already implements IComparable
    public int Compare(string x, string y)
    {
        return string.Compare(x, y, StringComparison.Ordinal);
    }
}

// Just use: strings.OrderBy(s => s) or strings.Sort()

## IEnumerable&lt;T&gt; and IEnumerator&lt;T&gt; Interfaces

### ✅ Good Example - Meaningful Custom Collection

In [ ]:
// ✅ GOOD: Custom collection with business logic and lazy evaluation
public class OrderHistory : IEnumerable<Order>
{
    private readonly List<Order> _orders = new List<Order>();
    private readonly int _customerId;
    
    public OrderHistory(int customerId)
    {
        _customerId = customerId;
    }
    
    public void AddOrder(Order order)
    {
        if (order.CustomerId != _customerId)
            throw new ArgumentException("Order belongs to different customer");
            
        _orders.Add(order);
    }
    
    // Business logic: Only enumerate orders from last 12 months by default
    public IEnumerator<Order> GetEnumerator()
    {
        var cutoffDate = DateTime.Now.AddMonths(-12);
        
        foreach (var order in _orders)
        {
            if (order.OrderDate >= cutoffDate)
                yield return order;
        }
    }
    
    // Get all orders including historical ones
    public IEnumerable<Order> GetAllOrders()
    {
        return _orders.AsReadOnly();
    }
    
    // Get orders by specific criteria with lazy evaluation
    public IEnumerable<Order> GetOrdersByAmount(decimal minAmount)
    {
        foreach (var order in _orders)
        {
            if (order.TotalAmount >= minAmount)
                yield return order;
        }
    }
    
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
}

public class Order
{
    public int OrderId { get; set; }
    public int CustomerId { get; set; }
    public DateTime OrderDate { get; set; }
    public decimal TotalAmount { get; set; }
    
    public Order(int orderId, int customerId, decimal totalAmount)
    {
        OrderId = orderId;
        CustomerId = customerId;
        OrderDate = DateTime.Now;
        TotalAmount = totalAmount;
    }
    
    public override string ToString() => $"Order {OrderId}: ${TotalAmount} on {OrderDate:yyyy-MM-dd}";
}

In [ ]:
// Usage example - business-aware enumeration
var customerOrders = new OrderHistory(123);

// Add some test orders
customerOrders.AddOrder(new Order(1, 123, 150.00m));
customerOrders.AddOrder(new Order(2, 123, 75.50m));
customerOrders.AddOrder(new Order(3, 123, 300.00m));

// Default enumeration shows only recent orders
Console.WriteLine("Recent orders (last 12 months):");
foreach (var order in customerOrders)
{
    Console.WriteLine(order);
}

// Use LINQ on our custom collection
var expensiveOrders = customerOrders.Where(o => o.TotalAmount > 100).ToList();
Console.WriteLine($"\nExpensive orders count: {expensiveOrders.Count}");

// Custom enumeration with business logic
Console.WriteLine("\nHigh-value orders (>= $200):");
foreach (var order in customerOrders.GetOrdersByAmount(200))
{
    Console.WriteLine(order);
}

// Can be used in LINQ operations
var totalSpent = customerOrders.Sum(o => o.TotalAmount);
Console.WriteLine($"\nTotal spent in last 12 months: ${totalSpent}");

### ✅ Good Example - Custom Iterator with Complex Logic

In [ ]:
// ✅ GOOD: Fibonacci sequence generator with business logic
public class FibonacciSequence : IEnumerable<long>
{
    private readonly int _maxCount;
    private readonly long _maxValue;
    
    public FibonacciSequence(int maxCount = int.MaxValue, long maxValue = long.MaxValue)
    {
        _maxCount = maxCount;
        _maxValue = maxValue;
    }
    
    public IEnumerator<long> GetEnumerator()
    {
        long current = 0;
        long next = 1;
        int count = 0;
        
        while (count < _maxCount && current <= _maxValue)
        {
            yield return current;
            count++;
            
            long temp = current + next;
            current = next;
            next = temp;
            
            // Prevent overflow in calculations
            if (next < 0) break;
        }
    }
    
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
}

In [ ]:
// Usage example - infinite sequence with business constraints
var fibonacci = new FibonacciSequence(maxCount: 15, maxValue: 1000);

Console.WriteLine("Fibonacci numbers (max 15 numbers, max value 1000):");
foreach (var number in fibonacci)
{
    Console.WriteLine(number);
}

// Can be used with LINQ
var evenFib = fibonacci.Where(n => n % 2 == 0).Take(5).ToArray();
Console.WriteLine($"\nFirst 5 even Fibonacci numbers: {string.Join(", ", evenFib)}");

// Lazy evaluation - only calculates what's needed
var firstLarge = fibonacci.FirstOrDefault(n => n > 100);
Console.WriteLine($"First Fibonacci > 100: {firstLarge}");

### ❌ Bad Example - Trivial IEnumerable Implementation

In [ ]:
// ❌ BAD: Just wrapping a List with no added value
public class BadStringCollection : IEnumerable<string>
{
    private List<string> _strings = new List<string>();
    
    public void Add(string item) => _strings.Add(item);
    
    // No business logic, just delegating to List
    public IEnumerator<string> GetEnumerator() => _strings.GetEnumerator();
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
}

// This adds no value - just use List<string> directly
// Or inherit from Collection<T> if you need a custom collection

## IDisposable Interface

### ✅ Good Example - Meaningful IDisposable Implementation

In [ ]:
// ✅ GOOD: Database connection manager with proper resource cleanup
public class DatabaseConnection : IDisposable
{
    private SqlConnection _connection;
    private bool _disposed = false;
    
    public DatabaseConnection(string connectionString)
    {
        _connection = new SqlConnection(connectionString);
        _connection.Open();
        Console.WriteLine("Database connection opened");
    }
    
    public void ExecuteQuery(string sql)
    {
        if (_disposed) throw new ObjectDisposedException(nameof(DatabaseConnection));
        
        using var command = new SqlCommand(sql, _connection);
        command.ExecuteNonQuery();
        Console.WriteLine($"Executed: {sql}");
    }
    
    public void Dispose()
    {
        Dispose(true);
        GC.SuppressFinalize(this);
    }
    
    protected virtual void Dispose(bool disposing)
    {
        if (!_disposed)
        {
            if (disposing)
            {
                // Dispose managed resources
                _connection?.Close();
                _connection?.Dispose();
                Console.WriteLine("Database connection closed and disposed");
            }
            
            // Free unmanaged resources here if any
            _disposed = true;
        }
    }
    
    ~DatabaseConnection()
    {
        Dispose(false);
    }
}

// Note: This is a simplified example. In real applications, use connection pooling
// and frameworks like Entity Framework that handle connections properly.

In [ ]:
// Usage example - automatic resource cleanup
using (var db = new DatabaseConnection("Server=localhost;Database=Test;"))
{
    db.ExecuteQuery("SELECT * FROM Users");
    db.ExecuteQuery("UPDATE Users SET LastLogin = GETDATE()");
    // Connection automatically closed when leaving using block
}

// Or with using declaration (C# 8.0+)
using var db2 = new DatabaseConnection("Server=localhost;Database=Test;");
db2.ExecuteQuery("SELECT COUNT(*) FROM Products");
// Connection closed at end of scope

## ICloneable Interface

### ✅ Good Example - Meaningful ICloneable Implementation

In [ ]:
// ✅ GOOD: Configuration object that needs deep cloning for different environments
public class ApplicationConfiguration : ICloneable
{
    public string Environment { get; set; }
    public DatabaseSettings Database { get; set; }
    public List<ApiEndpoint> ApiEndpoints { get; set; }
    public Dictionary<string, string> AppSettings { get; set; }
    
    public ApplicationConfiguration()
    {
        ApiEndpoints = new List<ApiEndpoint>();
        AppSettings = new Dictionary<string, string>();
    }
    
    // Deep clone for creating environment-specific configurations
    public object Clone()
    {
        var clone = new ApplicationConfiguration
        {
            Environment = Environment,
            Database = (DatabaseSettings)Database?.Clone(),
            ApiEndpoints = new List<ApiEndpoint>(),
            AppSettings = new Dictionary<string, string>(AppSettings)
        };
        
        // Deep clone the API endpoints
        foreach (var endpoint in ApiEndpoints)
        {
            clone.ApiEndpoints.Add((ApiEndpoint)endpoint.Clone());
        }
        
        return clone;
    }
    
    public ApplicationConfiguration CloneForEnvironment(string environment)
    {
        var clone = (ApplicationConfiguration)Clone();
        clone.Environment = environment;
        
        // Apply environment-specific modifications
        if (environment == "Production")
        {
            clone.Database.ConnectionString = clone.Database.ConnectionString.Replace("localhost", "prod-server");
            clone.AppSettings["LogLevel"] = "Error";
        }
        else if (environment == "Development")
        {
            clone.AppSettings["LogLevel"] = "Debug";
        }
        
        return clone;
    }
}

public class DatabaseSettings : ICloneable
{
    public string ConnectionString { get; set; }
    public int TimeoutSeconds { get; set; }
    public bool EnableRetry { get; set; }
    
    public object Clone() => new DatabaseSettings
    {
        ConnectionString = ConnectionString,
        TimeoutSeconds = TimeoutSeconds,
        EnableRetry = EnableRetry
    };
}

public class ApiEndpoint : ICloneable
{
    public string Name { get; set; }
    public string Url { get; set; }
    public int TimeoutMs { get; set; }
    
    public object Clone() => new ApiEndpoint
    {
        Name = Name,
        Url = Url,
        TimeoutMs = TimeoutMs
    };
}

In [ ]:
// Usage example - creating environment-specific configurations
var baseConfig = new ApplicationConfiguration
{
    Environment = "Base",
    Database = new DatabaseSettings 
    { 
        ConnectionString = "Server=localhost;Database=App;", 
        TimeoutSeconds = 30,
        EnableRetry = true
    },
    AppSettings = { ["LogLevel"] = "Info", ["CacheEnabled"] = "true" }
};

baseConfig.ApiEndpoints.Add(new ApiEndpoint { Name = "UserService", Url = "http://localhost:5001", TimeoutMs = 5000 });

// Create environment-specific configurations
var devConfig = baseConfig.CloneForEnvironment("Development");
var prodConfig = baseConfig.CloneForEnvironment("Production");

Console.WriteLine($"Base config environment: {baseConfig.Environment}");
Console.WriteLine($"Dev config environment: {devConfig.Environment}");
Console.WriteLine($"Prod config environment: {prodConfig.Environment}");

Console.WriteLine($"Base DB: {baseConfig.Database.ConnectionString}");
Console.WriteLine($"Prod DB: {prodConfig.Database.ConnectionString}");

Console.WriteLine($"Dev log level: {devConfig.AppSettings["LogLevel"]}");
Console.WriteLine($"Prod log level: {prodConfig.AppSettings["LogLevel"]}");

// Modifications to clones don't affect original
devConfig.Database.TimeoutSeconds = 60;
Console.WriteLine($"Base timeout: {baseConfig.Database.TimeoutSeconds}"); // Still 30
Console.WriteLine($"Dev timeout: {devConfig.Database.TimeoutSeconds}");   // Now 60

## Domain-Specific Real-World Examples

### E-Commerce Domain

In [ ]:
// Real-world example: Shopping cart with meaningful interface implementations
public class ShoppingCart : IEnumerable<CartItem>, IEquatable<ShoppingCart>
{
    private readonly List<CartItem> _items = new List<CartItem>();
    public string CartId { get; }
    public int CustomerId { get; }
    
    public ShoppingCart(string cartId, int customerId)
    {
        CartId = cartId ?? throw new ArgumentNullException(nameof(cartId));
        CustomerId = customerId;
    }
    
    public void AddItem(Product product, int quantity)
    {
        var existingItem = _items.FirstOrDefault(i => i.Product.Equals(product));
        if (existingItem != null)
        {
            existingItem.Quantity += quantity;
        }
        else
        {
            _items.Add(new CartItem(product, quantity));
        }
    }
    
    public decimal TotalAmount => _items.Sum(item => item.TotalPrice);
    public int TotalItems => _items.Sum(item => item.Quantity);
    
    // Meaningful enumeration: only active items
    public IEnumerator<CartItem> GetEnumerator()
    {
        return _items.Where(item => item.Quantity > 0).GetEnumerator();
    }
    
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
    
    // Business logic: Carts are equal if they belong to same customer
    public bool Equals(ShoppingCart other)
    {
        return other != null && CartId == other.CartId && CustomerId == other.CustomerId;
    }
    
    public override bool Equals(object obj) => Equals(obj as ShoppingCart);
    public override int GetHashCode() => HashCode.Combine(CartId, CustomerId);
}

public class CartItem
{
    public Product Product { get; }
    public int Quantity { get; set; }
    public decimal TotalPrice => Product.Price * Quantity;
    
    public CartItem(Product product, int quantity)
    {
        Product = product ?? throw new ArgumentNullException(nameof(product));
        Quantity = quantity;
    }
    
    public override string ToString() => $"{Product.Name} x{Quantity} = ${TotalPrice}";
}

In [ ]:
// Usage example - meaningful business operations
var cart = new ShoppingCart("cart-123", 456);

var laptop = new Product("LAP001", "Gaming Laptop", 1299.99m, "Electronics");
var mouse = new Product("MOU001", "Wireless Mouse", 29.99m, "Electronics");

cart.AddItem(laptop, 1);
cart.AddItem(mouse, 2);

Console.WriteLine($"Cart total: ${cart.TotalAmount} ({cart.TotalItems} items)");

// Use LINQ with our custom enumerable
Console.WriteLine("\nCart contents:");
foreach (var item in cart)
{
    Console.WriteLine(item);
}

// Business operations using interface implementations
var expensiveItems = cart.Where(item => item.TotalPrice > 100).ToList();
Console.WriteLine($"\nExpensive items: {expensiveItems.Count}");

// Cart comparison for session management
var cart2 = new ShoppingCart("cart-123", 456);
Console.WriteLine($"Same cart: {cart.Equals(cart2)}"); // True - same ID and customer

## Key Principles for Meaningful Interface Implementation

### When to Implement These Interfaces

**IEquatable&lt;T&gt;**:
- When objects have business identity (SKU, ID, etc.)
- For performance in collections (HashSet, Dictionary)
- When equality has domain-specific meaning

**IComparable&lt;T&gt;**:
- When objects have natural ordering in business context
- For automatic sorting in collections
- When comparison logic is complex or domain-specific

**IComparer&lt;T&gt;**:
- When you need multiple sorting strategies
- For complex sorting logic that doesn't belong in the class
- When working with types you can't modify

**IEnumerable&lt;T&gt;/IEnumerator&lt;T&gt;**:
- For custom collections with business logic
- When you need lazy evaluation or filtering
- For infinite or computed sequences

**IDisposable**:
- When managing unmanaged resources
- For expensive resources (connections, files)
- When implementing resource lifetime patterns

**ICloneable**:
- For creating modified copies without affecting originals
- When deep copying is needed for business scenarios
- For configuration or template patterns

### Anti-Patterns to Avoid

❌ **Don't implement interfaces just for the sake of it**
- Adding IEquatable to simple data containers
- Implementing IComparable with arbitrary sorting
- Creating IEnumerable wrappers around List&lt;T&gt;

❌ **Don't ignore business context**
- Equality should reflect business identity
- Comparison should have business meaning
- Enumeration should provide business value

❌ **Don't duplicate existing functionality**
- Use built-in interfaces and collections when possible
- Extend rather than replace standard behavior
- Add value through business logic

✅ **Do focus on business value**
- Implement interfaces to solve real problems
- Consider the consumer's perspective
- Design for the use cases you actually have